In [10]:
import torch
import os
import shutil

# --- 제공해주신 코드의 클래스들을 임포트합니다 ---

# 모델 관련
from models.model import (GATree, GATreePop, FEATURE_NUM, FEATURE_PAIR, ALL_FEATURES,
                   COL_NODE_TYPE, NODE_TYPE_UNUSED)

# 변이 연산자 관련
from evolution.Mutation.base import BaseMutation
from evolution.Mutation.chain import ChainMutation
from evolution.Mutation.node_param import NodeParamMutation
from evolution.Mutation.reinitialize_node import ReinitializeNodeMutation
from evolution.Mutation.add_node import AddNodeMutation
from evolution.Mutation.delete_node import DeleteNodeMutation
from evolution.Mutation.add_subtree import AddSubtreeMutation
from evolution.Mutation.delete_subtree import DeleteSubtreeMutation


# --- 테스트 환경 설정 ---
POP_SIZE = 1  # 하나의 트리에 집중하여 변화를 명확히 보기 위함
MAX_NODES = 128
MAX_DEPTH = 7
MAX_CHILDREN = 3
TEST_RESULTS_DIR = "test_results"

# --- 테스트에 필요한 설정(config) 딕셔너리 ---
# 변이 연산자들이 트리의 제약조건을 알아야 하므로 config가 필요합니다.
config = {
    'max_nodes': MAX_NODES,
    'max_depth': MAX_DEPTH,
    'max_children': MAX_CHILDREN,
    'feature_num': FEATURE_NUM,
    'feature_pair': FEATURE_PAIR,
    'all_features': ALL_FEATURES,
}

In [11]:
# 1. 테스트를 위한 초기 집단 생성
print("===== [Setup] Creating initial GATree population... =====")
initial_population = GATreePop(POP_SIZE, MAX_NODES, MAX_DEPTH, MAX_CHILDREN, FEATURE_NUM, FEATURE_PAIR)
initial_population.make_population()
print("Population created successfully.")

# 1. 테스트 전 상태 저장 (깊은 복사)
original_tensor = initial_population.population_tensor.clone()

# 원본 GATree 객체 생성 (시각화용)
tree_before = GATree(MAX_NODES, MAX_DEPTH, MAX_CHILDREN, FEATURE_NUM, FEATURE_PAIR,
                        data_tensor=original_tensor[0])

tree_after = GATree(MAX_NODES, MAX_DEPTH, MAX_CHILDREN, FEATURE_NUM, FEATURE_PAIR,
                        data_tensor=original_tensor[0])

===== [Setup] Creating initial GATree population... =====
--- Creating Tree 1/1 ---
Tree created with 28 nodes.

Population created successfully.
Population created successfully.


In [12]:
tree_before.load('/Users/yoon-seunggyu/Documents/GA_Tree/GA_tree/test_results/DeleteNodeMutation_before.pth')
tree_after.load('/Users/yoon-seunggyu/Documents/GA_Tree/GA_tree/test_results/DeleteNodeMutation_after.pth')

Tree loaded successfully.
Tree loaded successfully.


In [15]:
tree_before.data[41]

tensor([ 3.0000, 36.0000,  5.0000,  1.0000,  0.8448, 27.0000,  0.0000])

In [16]:
tree_after.data[41]

tensor([ 3.0000, 36.0000,  5.0000,  1.0000,  0.8448, 27.0000,  0.0000])

In [17]:
tree_before.next_idx

42

In [18]:
tree_after.next_idx

41

In [24]:
import random

# 3. 현재 포지션에 따라 예측 실행
print("\n3. Running predictions based on current position...")

# Case 2: 현재 포지션이 'SHORT'일 경우
current_pos_short = 'SHORT'
action_short = tree_after.predict({}, current_pos_short)
print(f"-> When current position is '{current_pos_short}', Predicted Action: {action_short}")


print("\nPrediction Demo Finished.")
print("Check 'prediction_demo_tree.html' to trace the prediction paths.")


3. Running predictions based on current position...
-> When current position is 'SHORT', Predicted Action: ('LONG', 0.5229098200798035, 76)

Prediction Demo Finished.
Check 'prediction_demo_tree.html' to trace the prediction paths.


In [23]:
tree_after.next_idx = 9